<br><br>

## **Import necessary Python libraries and modules**

In [1]:

# For data manipulation and analysis
import pandas as pd
import numpy as np

# For machine learning tools and evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report


# For deep learning
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
import torch



To use the HuggingFace [`transformers` Python library](https://huggingface.co/transformers/installation.html), we will install it with `pip`.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 10.7 MB/s 
     |████████████████████████████████| 895 kB 48.2 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 
     |████████████████████████████████| 6.5 MB 46.9 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Once `transformers` is installed, we will import modules for `DistilBert`, a *distilled* or smaller version of a BERT model that runs more quickly and uses less computing power. This makes it ideal for those just getting started with BERT.

In [3]:
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

<br><br>

## **Set parameters and file paths**

In [4]:
# This is the name of the BERT model that we want to use. 
# We're using DistilBERT to save space (it's a distilled version of the full BERT model), 
# and we're going to use the cased (vs uncased) version.
model_name = 'distilbert-base-multilingual-cased'  

# This is the name of the program management system for NVIDIA GPUs. We're going to send our code here.
device_name = 'cuda'       

# This is the maximum number of tokens in any document sent to BERT.
max_length = 128                                                       

# This is the name of the directory where we'll save our model. You can name it whatever you want.
#cached_model_directory_name = 'ABSA_FineTuning_BERT'  
cached_model_directory_name = 'Emotions_dev2'  

In [5]:
#stiamo utlizzando la GPU?
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [6]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 7.0 MB/s 
     |████████████████████████████████| 448 kB 37.9 MB/s 


In [7]:
#pretokenizzazionen in morfemi delle koreane
from konlpy.tag import Okt
def pretok_ko(subset, indice):
  new_ko=[] 
  #count=0
  for index,row in subset.iterrows():
    if index < indice:
      new_ko.append(row['Sentence'])
    else: #quando arrivi all'indice di ko

      okt = Okt()
      li=okt.morphs(str(row['Sentence']))
      stringa=''
      for el in li:
        stringa+=str(el)+' '
      new_ko.append(stringa[:-1])
  subset['Sentence']=new_ko
  return subset
    


In [8]:
dev=pd.read_csv("dev_def.csv")
dev=pretok_ko(dev,2993)
dev

,Unnamed: 0,level_0,Sentence,Emotions
0,0,2,That definition has changed in the last couple...,Neutral
1,1,3,"""Cult"" now signifies a deliberate weirdness an...",Neutral
2,2,15,He believes Santa punishes the bad and starts ...,Neutral
3,3,50,It will stay with you for a long time.,Neutral
4,4,72,Let alone that these things ALSO happen in rea...,Neutral
...,...,...,...,...
4743,4743,838,쌍둥이 가 의사 소통 에 사용 하는 언어 인 ' twinnish ' 가 많은 Dia...,Neutral
4744,4744,865,그리고 유럽 여러 도시 를 여행 해야 하지만 요르단 과 같은 건물 에 있는 작은 아...,Neutral
4745,4745,869,Arthur Penhaligon 은 곧 죽 을 것 입니다 . 미스터 먼데이 와 그 ...,Neutral
4746,4746,871,& lt ; br & gt ; 이 소설 은 쌍둥이 에 관 한 이야기 입니다 .,Neutral


In [9]:
test=pd.read_csv("test_def.csv")
test=pretok_ko(test,5981)
test

,Unnamed: 0,level_0,Sentence,Emotions
0,0,33476,This does not sound like a good situation for ...,Anger
1,1,11870,There's really no way to know since my lady ho...,Neutral
2,2,34327,Considering that cheating is a mark of high-st...,Anger
3,3,50190,i am feeling a little apprehensive but i m sur...,Fear
4,4,49266,i still feel like there are more than enough t...,Joy
...,...,...,...,...
9484,9484,92647,"멋있을 수도 있는 게 하나도 안 멋있고, 감동적일 수도 있는 장면에 아무런 감동도 ...",Surprise
9485,9485,92038,배우가 너무 아깝다고 생각했다.,Surprise
9486,9486,65497,음 기왕 일산 까지 보신 다면 렉서스 의 suv 하브 모델 은 어떠하실 지요 . R...,Neutral
9487,9487,45413,발연기. 득보잡 배우들. 장르는 저예산 에로.,Fear


In [10]:
#creiamo le liste di train e test. 1.8k train 250 test  
train_texts=dev['Sentence']
train_labels=dev['Emotions']
test_texts=test['Sentence']
test_labels=test['Emotions']


<br><br>

## **Split the data into training and test sets**

In [11]:
len(train_texts), len(train_labels), len(test_texts), len(test_labels)

(4748, 4748, 9489, 9489)

Here's an example of a training label and review:

In [12]:
train_labels[0], train_texts[0]

('Neutral', 'That definition has changed in the last couple of decades.')

<br><br>

## **Encode data for BERT**

We're going to transform our texts and labels into a format that BERT (via Huggingface and PyTorch) will understand. This is called *encoding* the data.

Here are the steps we need to follow:

1. The labels&mdash;in this case, Goodreads genres&mdash;need to be turned into integers rather than strings.

2. The texts&mdash;in this case, Goodreads reviews&mdash;need to be truncated if they're more than 512 tokens or padded if they're fewer than 512 tokens. The tokens, or words in the texts, also need to be separated into "word pieces" and matched to their embedding vectors.

3. We need to add special tokens to help BERT:

| BERT special token | Explanation |
| --------------| ---------|
| [CLS] | Start token of every document. |
| [SEP] | Separator between each sentence |
| [PAD] | Padding at the end of the document as many times as necessary, up to 512 tokens |
|  &#35;&#35; | Start of a "word piece" |




Here we will load `DistilBertTokenizerFast` from the HuggingFace library, which will do all the work of encoding the texts for us. The `tokenizer()` will break word tokens into word pieces, truncate to 512 tokens, and add padding and special BERT tokens.

In [13]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name) # The model_name needs to match our pre-trained model.

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Here we will create a map of our labels, or Goodreads genres, to integer keys. We take the unique labels, and then we make a dictionary that associates each label/tag with an integer.

**Note:** HuggingFace documentation sometimes refers to "labels" as "tags" but these are the same thing. We use "labels" throughout this notebook for clarity.

In [ ]:
#unique_labels = set(label for label in train_labels)
#label2id = {label: id for id, label in enumerate(unique_labels)}
#id2label = {id: label for label, id in label2id.items()}

In [14]:
unique_labels={'Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger','Fear'}
label2id={'Sadness':0, 'Surprise':1, 'Joy':2, 'Neutral':3, 'Anger':4,'Fear':5}
id2label={ 0:'Sadness', 1:'Surprise', 2:'Joy', 3:'Neutral', 4:'Anger',5:'Fear'}

In [15]:
unique_labels

{'Anger', 'Fear', 'Joy', 'Neutral', 'Sadness', 'Surprise'}

In [16]:
label2id.keys()

dict_keys(['Sadness', 'Surprise', 'Joy', 'Neutral', 'Anger', 'Fear'])

In [17]:
id2label.keys()

dict_keys([0, 1, 2, 3, 4, 5])

Now let's encode our texts and labels!

In [18]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=128)
test_encodings  = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=128)

train_labels_encoded = [label2id[y] for y in train_labels.tolist()]
test_labels_encoded  = [label2id[y] for y in test_labels.tolist()]

In [19]:
type(test_labels_encoded[100])

int

In [20]:
' '.join(train_encodings[29].tokens[0:100])

'[CLS] Was that supposed to be cut ##e ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [21]:
' '.join(test_encodings[0].tokens[0:100])

'[CLS] This does not sound like a good situation for a child at all im ##o . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [22]:
set(train_labels_encoded)

{0, 1, 2, 3, 4, 5}

In [23]:
set(test_labels_encoded)

{0, 1, 2, 3, 4, 5}

<br><br>

## **Make a custom Torch dataset**

Here we combine the encoded labels and texts into dataset objects. We use the custom Torch `MyDataSet` class to make a `train_dataset` object from  the `train_encodings` and `train_labels_encoded`. We also make a `test_dataset` object from `test_encodings`, and `test_labels_encoded`.

In [24]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [25]:
train_dataset = MyDataset(train_encodings, train_labels_encoded)
test_dataset = MyDataset(test_encodings, test_labels_encoded)

In [26]:
' '.join(train_dataset.encodings[0].tokens[0:100])

'[CLS] That definition has changed in the last couple of decades . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [27]:
' '.join(test_dataset.encodings[1].tokens[0:100])

"[CLS] There ' s really no way to know since my lady hormone ##s prevent me from remember ##ing sexual assault ##ers ' ide ##nti ##ties . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

<br><br>

## **Load pre-trained BERT model**

Here we load a pre-trained DistilBERT model and send it to CUDA.

**Note:** If you decide to repeat fine-tuning after already running the following cells, make sure that you re-run this cell to re-load the original pre-trained model before fine-tuning again.

In [28]:
# The model_name needs to match the name used for the tokenizer above.
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=len(id2label)).to(device_name)

Downloading:   0%|          | 0.00/517M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'pre_classif

<br><br>

## **Set the BERT fine-tuning parameters**

These are the arguments we'll set in the HuggingFace TrainingArguments objects, which we'll then pass to the HuggingFace Trainer object. There are many more possible arguments, but here we highlight the basics and some common gotchas.

When training your own model, you should search over these parameters to find the best settings for your particular dataset. You should use a held-out set of validation data for this step.

| Parameter | Explanation |
|-----------| ------------|
| num_train_epochs | total number of training epochs (how many times to pass through the entire dataset; too much can cause overfitting) |
| per_device_train_batch_size | batch size per device during training |
| per_device_eval_batch_size |  batch size for evaluation |
|  warmup_steps |  number of warmup steps for learning rate scheduler (set lower because of small dataset size) |
| weight_decay | strength of weight decay (reduces size of weights, like regularization) |
| output_dir | output directory for the fine-tuned model and configuration files |
| logging_dir | directory for storing logs |
| logging_steps | how often to print logging output (so that we can stop training early if the loss isn't going down) |
| evaluation_strategy | evaluate while training so that we can see the accuracy going up |

In [32]:
training_args = [TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'
    # evaluate during fine-tuning so that we can see progress
), TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
), 
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
), 
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=2e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
),
TrainingArguments(
    num_train_epochs=4,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    learning_rate=3e-5,              # initial learning rate for Adam optimizer
    warmup_steps=400,                # number of warmup steps for learning rate scheduler (set lower because of small dataset size)
    weight_decay=0.01,               # strength of weight decay
    output_dir='./results',          # output directory
    logging_dir='./logs',            # directory for storing logs
    logging_steps=400,               # number of steps to output logging (set lower because of small dataset size)
    evaluation_strategy='steps',
    save_strategy='epoch'# evaluate during fine-tuning so that we can see progress
)]

using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
using `logging_steps` to initialize `eval_steps` to 400
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should sta

<br><br>

## **Fine-tune the BERT model**

First, we define a custom evaluation function that returns the accuracy. You could modify this function to return precision, recall, F1, and/or other metrics.

In [30]:
def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
  }

Then we create a HuggingFace `Trainer` object using the `TrainingArguments` object that we created above. We also send our `compute_metrics` function to the `Trainer` object, along with our test and train datasets.

**Note:** This is what we've been aiming for this whole time! All the work of tokenizing, creating datasets, and setting the training arguments was for this cell.

In [33]:
for argomenti_train in training_args:
  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=argomenti_train,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset (usually a validation set; here we just send our test set)
    compute_metrics=compute_metrics      # our custom evaluation function 
    )
  
  trainer.train()

  trainer.evaluate()   

  predicted_results = trainer.predict(test_dataset)

  predicted_labels = predicted_results.predictions.argmax(-1) # Get the highest probability prediction
  predicted_labels = predicted_labels.flatten().tolist()      # Flatten the predictions into a 1D list
  predicted_labels = [id2label[l] for l in predicted_labels]  # Convert from integers back to strings for readability

  print(classification_report(test_labels, 
                            predicted_labels))
  print('\n')


#meglio prediligere learn rate bassi ad ogni modo
#https://arxiv.org/pdf/1905.05583.pdf

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4748
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 447


Step,Training Loss,Validation Loss,Accuracy
400,0.795700,1.561703,0.494678


Saving model checkpoint to ./results/checkpoint-149
Configuration saved in ./results/checkpoint-149/config.json
Model weights saved in ./results/checkpoint-149/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-298
Configuration saved in ./results/checkpoint-298/config.json
Model weights saved in ./results/checkpoint-298/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 9489
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-447
Configuration saved in ./results/checkpoint-447/config.json
Model weights saved in ./results/checkpoint-447/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9489
  Batch size = 32


***** Running Prediction *****
  Num examples = 9489
  Batch size = 32
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 4748
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 447


              precision    recall  f1-score   support

       Anger       0.47      0.39      0.42      1547
        Fear       0.48      0.51      0.50      1497
         Joy       0.48      0.51      0.50      1606
     Neutral       0.68      0.58      0.62      1910
     Sadness       0.47      0.41      0.44      1457
    Surprise       0.39      0.52      0.45      1472

    accuracy                           0.49      9489
   macro avg       0.49      0.49      0.49      9489
weighted avg       0.50      0.49      0.49      9489





...

              precision    recall  f1-score   support

       Anger       0.46      0.42      0.44      1547
        Fear       0.53      0.48      0.51      1497
         Joy       0.49      0.51      0.50      1606
     Neutral       0.68      0.56      0.62      1910
     Sadness       0.44      0.56      0.49      1457
    Surprise       0.47      0.52      0.49      1472

    accuracy                           0.51      9489
   macro avg       0.51      0.51      0.51      9489
weighted avg       0.52      0.51      0.51      9489





Step,Training Loss,Validation Loss,Accuracy
400,0.017200,4.211249,0.485615
800,0.068900,3.936783,0.507430


Saving model checkpoint to ./results/checkpoint-297
Configuration saved in ./results/checkpoint-297/config.json
Model weights saved in ./results/checkpoint-297/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 9489
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-594
Configuration saved in ./results/checkpoint-594/config.json
Model weights saved in ./results/checkpoint-594/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 9489
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-891
Configuration saved in ./results/checkpoint-891/config.json
Model weights saved in ./results/checkpoint-891/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 9489
  Batch size = 16


***** Running Prediction *****
  Num examples = 9489
  Batch size = 16


              precision    recall  f1-score   support

       Anger       0.47      0.42      0.44      1547
        Fear       0.52      0.51      0.52      1497
         Joy       0.49      0.51      0.50      1606
     Neutral       0.67      0.58      0.62      1910
     Sadness       0.46      0.52      0.49      1457
    Surprise       0.47      0.52      0.49      1472

    accuracy                           0.51      9489
   macro avg       0.51      0.51      0.51      9489
weighted avg       0.52      0.51      0.51      9489





In [ ]:
#anche qui il compound 5 è top